In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
from scipy import ndimage
import scipy.io
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from lib import loader, modules

## Create results directory

In [ ]:
try: 
    os.mkdir('results') 
except OSError as error: 
    print(error)

## Simulation type

In [ ]:
simName = "DPM" # Options: DPM, ZSDPMtoIRT2, DPMtoIRT2, DPMcars, IRT2carsCDPM, IRT2carsCDPMtoIRT, 
                # from top to bottom in Table II of the paper.

## Log file to save the best validation loss among the models of 50 training epochs

In [ ]:
with open('results/' + simName + 'Log.txt', 'w') as f:
    print('Training Accuracy', file=f)
    print('-' * 20, file=f)

## Loaders

In [ ]:
Loc_train = loader.locDL(phase="train",dir_dataset="dataset/",cityMap="true",carsMap="false",simulation=simName,TxMaps="true")
Loc_val = loader.locDL(phase="val",dir_dataset="dataset/",cityMap="true",carsMap="false",simulation=simName,TxMaps="true") 

In [ ]:
dataloaders = {
    'train': DataLoader(Loc_train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True),
    'val': DataLoader(Loc_val, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
}

# Training

In [ ]:
torch.set_default_dtype(torch.float32)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

batch_size = 15

inp = 16
model = modules.LocUNet(inputs=inp)
model.cuda()


import torch.optim as optim
from torch.optim import lr_scheduler
import time
from datetime import datetime
import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.nn as nn

def my_loss(output, target):
    loss = torch.sum((output - target)**2,1)
    loss = torch.sqrt(loss)
    loss = torch.mean(loss)
    return loss


def calc_loss_dense(pred, target, metrics):
    loss = my_loss(pred, target)# *256*256
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    return loss



def print_metrics(metrics, epoch_samples, phase):
    outputs1 = []
    for k in metrics.keys():
        outputs1.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
    with open('results/Log.txt', 'a') as f:
        print("{}: {}".format(phase, ", ".join(outputs1)), file=f)

def train_model(model, optimizer, scheduler, num_epochs=50):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    for epoch in range(num_epochs):
        with open('results/Log.txt', 'a') as f:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1), file=f)
            print('-' * 10, file=f)

        since = time.time()
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    with open('results/Log.txt', 'a') as f:
                        print("learning rate", param_group['lr'], file=f)

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, targets in dataloaders[phase]:
                inputs = inputs.to(device)
                targets = targets.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs1 = model(inputs)
                    loss = calc_loss_dense(outputs1.float(), targets.float(), metrics)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # track number of samples in the mini-batch
                epoch_samples += inputs.size(0)
            

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        with open('results/Log.txt', 'a') as f:
            print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), file=f)
            now = datetime.now()
            print("now =", now, file=f)
    with open('results/Log.txt', 'a') as f:
        print('Best val loss: {:4f}'.format(best_loss), file=f)
   
    # Return the best model weights according to the validation loss
    model.load_state_dict(best_model_wts)
    return model

### Execute training

In [ ]:
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)
model = train_model(model, optimizer_ft, exp_lr_scheduler)

### Save the model with the best validation loss

In [ ]:
stringer = 'results/' + simName + 'BestModel.pt'
torch.save(model.state_dict(), stringer)

# Test

### Load the model with the best validation loss

In [ ]:
stringer = 'results/' + simName + 'BestModel.pt'
model.load_state_dict(torch.load(stringer))
model.cuda()

## Log file to save the test loss 

In [ ]:
with open('results/' + simName + 'LogTest.txt', 'w') as f:
    print('Test Accuracy', file=f)
    print('-' * 20, file=f)

## Test Loop

In [ ]:
import torch
import time
from datetime import datetime
import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.nn as nn

batch_sizeTest = 1


def my_loss(output, target):
    loss = torch.sum((output - target)**2,1)
    loss = torch.sqrt(loss)
    loss = torch.mean(loss)
    return loss


def calc_loss_test(pred, target, metrics):
    loss = my_loss(pred, target)# *256*256
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    return loss


def print_metrics(metrics, epoch_samples, phase):
    outputs1 = []
    for k in metrics.keys():
        outputs1.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
    with open('results/' + simName + 'LogTest.txt', 'a') as f:
        print("{}: {}".format(phase, ", ".join(outputs1)), file=f)
        
def test_loss(model):
    since = time.time()
    model.eval()   # Set model to evaluate mode
    metrics = defaultdict(float)
    epoch_samples = 0
            
    for inputs, targets in DataLoader(Loc_test, batch_size=batch_sizeTest, shuffle=True, num_workers=8):
        inputs = inputs.to(device)
        targets = targets.to(device)
        with torch.set_grad_enabled(False):
            outputs1 = model(inputs)
            loss = calc_loss_test(outputs1.float(), targets.float(), metrics)
            epoch_samples += inputs.size(0)
     
    print_metrics(metrics, epoch_samples, phase='test')

    time_elapsed = time.time() - since
    with open('results' + simName + '/LogTest.txt', 'a') as f:
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), file=f)
        now = datetime.now()
        print("now =", now, file=f)

### Execute test

In [ ]:
test_loss(model)

# Generate results as images

### Load the model with the best validation loss

In [ ]:
stringer = 'results/' + simName + 'BestModel.pt'
model.load_state_dict(torch.load(stringer))
model.cuda()

In [ ]:
from lib import loader, modules, modulesHeatMapOut

### Copy parameters to NN with heatmap output

In [ ]:
model_dict = model.state_dict()
inp = 16
modelMapOut = modulesHeatMapOut.LocUNet(inputs=inp)
modelMapOut_dict = modelMapOut.state_dict()

# Taken from https://discuss.pytorch.org/t/how-to-load-part-of-pre-trained-model/1113
#Now copy and print        
pretrained_dict = model_dict
model2_dict = modelMapOut_dict
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model2_dict}
# 2. overwrite entries in the existing state dict
model2_dict.update(pretrained_dict)
# 3. load the new state dict
modelMapOut.load_state_dict(model2_dict)

modelMapOut.cuda()

### Create directory for the images

In [ ]:
try: 
    os.mkdir('results/img') 
except OSError as error: 
    print(error)
    
try: 
    os.mkdir('results/img/' + simName) 
except OSError as error: 
    print(error)

### Save the images

In [ ]:
import cv2
from torch import linalg as LA
maps_inds=np.arange(0,99,1,dtype=np.int16)
#Standard determenistic "random" shuffle of the maps:
np.random.seed(42)
np.random.shuffle(maps_inds)


noTrials = 50 # No of Rx considered for each map with one fixed BS deployment, max. 200

from PIL import Image
from numpy.linalg import norm
for mapp in range(15):
    name00=str(mapp)
    name0=str(maps_inds[84+mapp])
    Loc_test2 = loader.locDL(maps_inds=maps_inds, phase="custom", dir_dataset="dataset/",simulation=simName,
                                           cityMap="true",carsMap="false",TxMaps="true",
                                           ind1=84+mapp,ind2=84+mapp
                                           )
    
    ii=0
    for inputs, target in DataLoader(Loc_test2, batch_size=1, shuffle=False, num_workers=0):
     
        if ii>noTrials-1:
            break
            
        mapEst, est = modelMapOut(inputs.cuda())
        
 
        inputss=inputs.detach().cpu().numpy()
        targets=target.detach().cpu().numpy()
        
        
        mapEsts = mapEst.detach().cpu().numpy()
        ests=est.detach().cpu().numpy()

        mapEsts = mapEsts - np.min(mapEsts[0][0])
        mapEsts = mapEsts/np.max(mapEsts[0][0])
        mapEsts = 255*mapEsts


        builds=inputss[0][15]
        indB=builds!=0
        im=np.zeros([256,256,3])

        im[:,:,0] = (mapEsts[0][0])
        im[:,:,1] = (mapEsts[0][0])
        im[:,:,2] = (mapEsts[0][0])

        name=str(ii)

        im[indB,2]=200
        im[indB,0]=0
        im[indB,1]=0

        im = Image.fromarray(im.astype(np.uint8)) 

        file_name="results/img/" + simName + "/" + name00 + "_" + name0 + "_" + name + ".png" 
        
        im.save(file_name)
        
        
        img = cv2.imread(file_name)
        
        aa = np.float(255)

        cv2.drawMarker(img, (targets[0][1],targets[0][0]), color=(0,255,0), markerType=cv2.MARKER_CROSS, thickness=2)
        cv2.drawMarker(img, (ests[0][1],ests[0][0]), color=(0,255,255), markerType=cv2.MARKER_SQUARE, thickness=2)
        for tx in range(5):
            imTx = inputss[0][10+tx]
            ind = np.unravel_index(np.argmax(imTx, axis=None), imTx.shape)
            cv2.drawMarker(img, (ind[1], ind[0]), color=(0,0,255), markerType=cv2.MARKER_DIAMOND, thickness=2)
              
        cv2.imwrite(file_name,img)
        ii=ii+1
